1. Add the "OwnerParcel" layer from the parcels GDB for the city
2. Add the zoning layer

In [50]:
city = 'Missoula'
zone_field = 'Zoning'
source_parcels = r'shp\Missoula_GDB\Missoula_Parcels.gdb\OwnerParcel'
source_zoning = r'shp\zoning.gdb\missoula'

import os
import shutil
gdb = os.path.join('shp', city + '.gdb')
spatial_join_layer = "OwnerParcel_SpatialJoin_" + city

In [2]:
# if baseline GDB exists, delete it
if os.path.isdir(gdb):
    shutil.rmtree(gdb)

arcpy.management.CreateFileGDB('shp', city)

<Result 'shp\\Missoula.gdb'>

In [3]:
arcpy.conversion.FeatureClassToFeatureClass(
    source_parcels,
    gdb,
    "OwnerParcel"
)

<Result 'shp\\Missoula.gdb\\OwnerParcel'>

In [4]:
arcpy.conversion.FeatureClassToFeatureClass(
    source_zoning,
    gdb,
    "Zoning"
)

<Result 'shp\\Missoula.gdb\\Zoning'>

In [2]:
arcpy.analysis.SpatialJoin(
    "OwnerParcel", 
    "Zoning", 
    os.path.join(gdb, spatial_join_layer), 
    "JOIN_ONE_TO_ONE", 
    "KEEP_COMMON", 
    None, 
    "HAVE_THEIR_CENTER_IN")

<Result 'Z:\\projects\\zoning_atlas\\shp\\Missoula.gdb\\OwnerParcel_SpatialJoin_Missoula'>

In [6]:
arcpy.management.SelectLayerByAttribute(
    spatial_join_layer, 
    "NEW_SELECTION", 
    """
    Zoning IN ('R215', 'R80', 'R40', 'R20', 'RT10', 'R8', 
    'R5.4', 'RT5.4', 'R3', 'RT2.7', 'RM2.7', 'RM2', 'RM1.5', 
    'RM1', 'RM0.5', 'RMH') 
    AND PARCELID IS NOT NULL 
    AND PropType NOT IN (
        '',
        'VR - Vacant Land Rural', 
        'CA - Centrally Assessed', 
        'VU - Vacant Land Urban',
        'FARM_U - Farmstead - Urban', 
        'NVS - Non-Valued with Specials', 
        'RV_PARK - RV Park', 
        'GRAVEL - Gravel Pit', 
        'GOLF - Golf Course',
        'EP_PART - Partial Exempt Property', 
        'CN - Centrally Assessed Non-Valued Property', 
        'NV - Non-Valued Property', 'FARM_R - Farmstead - Rural', 
        'VAC_U - Vacant Land - Urban', 
        'EP - Exempt Property', 'VAC_R - Vacant Land - Rural'
    )
    """, 
    "NON_INVERT"
)

id,value
0,a Layer object
1,12935


In [7]:
arcpy.conversion.FeatureClassToFeatureClass(
    spatial_join_layer, 
    gdb, 
    "residential_parcels"
)
# then rename to residential_parcels

<Result 'Z:\\projects\\zoning_atlas\\shp\\Missoula.gdb\\residential_parcels'>

In [8]:
arcpy.management.CalculateGeometryAttributes(
    "residential_parcels", 
    "lot_size AREA", 
    '', 
    "SQUARE_FEET_US", 
    None, 
    "SAME_AS_INPUT"
)

<Result 'residential_parcels'>

In [9]:
arcpy.management.Project(
    "residential_parcels", 
    os.path.join(gdb, 'parcels_ft'), 
    'PROJCS["NAD_1983_StatePlane_Montana_FIPS_2500_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Foot_US",0.3048006096012192]]', "'WGS_1984_(ITRF08)_To_NAD_1983_2011 + WGS_1984_(ITRF00)_To_NAD_1983'", 'PROJCS["NAD_1983_2011_StatePlane_Montana_FIPS_2500",GEOGCS["GCS_NAD_1983_2011",DATUM["D_NAD_1983_2011",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",600000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-109.5],PARAMETER["Standard_Parallel_1",45.0],PARAMETER["Standard_Parallel_2",49.0],PARAMETER["Latitude_Of_Origin",44.25],UNIT["Meter",1.0]]', 
    "NO_PRESERVE_SHAPE", 
    None, 
    "NO_VERTICAL"
)

<Result 'Z:\\projects\\zoning_atlas\\shp\\Missoula.gdb\\parcels_ft'>

In [10]:
arcpy.management.MinimumBoundingGeometry(
    os.path.join(gdb, 'parcels_ft'), 
    os.path.join(gdb, 'parcels_mbb'), 
    "RECTANGLE_BY_WIDTH", 
    "NONE", 
    None, 
    "MBG_FIELDS"
)

<Result 'Z:\\projects\\zoning_atlas\\shp\\Missoula.gdb\\parcels_mbb'>

In [11]:
arcpy.management.JoinField(
    "residential_parcels", 
    'PARCELID', 
    os.path.join(gdb, 'parcels_mbb'), 
    'PARCELID', 
    'MBG_Width;MBG_Length'
)

<Result 'residential_parcels'>

In [12]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "city", 
    '"' + city + '"'
)

<Result 'residential_parcels'>

In [12]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "zoning", 
    "!" + zone_field + "!", 
    "PYTHON3"
)

<Result 'residential_parcels'>

In [51]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_size", 
    "round(!lot_size!)", 
    "PYTHON3", 
    None, 
    "SHORT"
)

<Result 'residential_parcels'>

In [16]:
arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_width", 
    "!MBG_Width!", 
    "PYTHON3", 
    None, 
    "SHORT"
)

<Result 'residential_parcels'>

In [19]:
codeblock = """
def eval_zoning(zone):
    if zone in ['R215', 'R80', 'R40', 'R20', 'R8', 'R5.4']:
        return 1
    elif zone in ['RT10', 'RT5.4', 'RT2.7']:
        return 2
    elif zone in ['RM2.7', 'RM2', 'RM1.5', 'RM1', 'RM0.5', 'RMH']:
        return 5
    else:
        return 0
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "zoning_allows", 
    "eval_zoning(!zoning!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [28]:
codeblock = """
def eval_zoning(zone, lot_size):
    if zone == 'RT-10' and lot_size >= 20000:
        return 2
    elif zone == 'RT5.4' and lot_size >= 10800:
        return 2
    elif zone == 'RT2.7' and lot_size >= 5400:
        return 2
    elif zone == 'RM2.7':
        return min(max(1, math.floor(lot_size / 2700)), 5)
    elif zone == 'RM2':
        return min(max(1, math.floor(lot_size / 2000)), 5)
    elif zone == 'RM1.5':
        return min(max(1, math.floor(lot_size / 1500)), 5)
    elif zone in ['RM1', 'RHM']:
        return min(max(1, math.floor(lot_size / 1000)), 5)
    elif zone == 'RM0.5':
        return min(max(1, math.floor(lot_size / 500)), 5)
    else:
        return 1
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_size_allows", 
    "eval_zoning(!zoning!, !lot_size!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [26]:
codeblock = """
def eval_zoning(zone, lot_width):
    return 5
"""

arcpy.management.CalculateField(
    "residential_parcels", 
    "lot_width_allows", 
    "eval_zoning(!zoning!, !lot_width!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'residential_parcels'>

In [52]:
codeblock = """
def units(zone, size, width):
    return max(1, min(zone, size, width))
"""
arcpy.management.CalculateField(
    "residential_parcels", 
    "units_allowed", 
    "units(!zoning_allows!, !lot_size_allows!, !lot_width_allows!)",
    "PYTHON3",
    codeblock,
    "SHORT"
)

<Result 'residential_parcels'>

In [53]:
codeblock = """
def categorize(zone):
    if zone in ['RT10', 'RT5.4', 'RT2.7', 'RM2.7',
    'RM2', 'RM1.5', 'RM1', 'RM0.5', 'RMH']:
        return "Penalized Multifamily"
    elif zone in []:
        return "Multifamily"
    else:
        return "Single-Family"
"""
arcpy.management.CalculateField(
    "residential_parcels", 
    "category", 
    "categorize(!zoning!)",
    "PYTHON3",
    codeblock,
    "TEXT"
)

<Result 'residential_parcels'>

In [54]:
arcpy.conversion.FeatureClassToFeatureClass(
    "residential_parcels", 
    gdb, 
    city.lower() + "_residential_parcels", 
    '', 
    'city "city" true true false 512 Text 0 0,First,#,residential_parcels,city,0,512;zoning "zoning" true true false 512 Text 0 0,First,#,residential_parcels,zoning,0,512;lot_size "lot_size" true true false 8 Double 0 0,First,#,residential_parcels,lot_size,-1,-1;lot_width "lot_width" true true false 2 Short 0 0,First,#,residential_parcels,lot_width,-1,-1;zoning_allows "zoning_allows" true true false 2 Short 0 0,First,#,residential_parcels,zoning_allows,-1,-1;lot_size_allows "lot_size_allows" true true false 2 Short 0 0,First,#,residential_parcels,lot_size_allows,-1,-1;lot_width_allows "lot_width_allows" true true false 2 Short 0 0,First,#,residential_parcels,lot_width_allows,-1,-1;units_allowed "units_allowed" true true false 2 Short 0 0,First,#,residential_parcels,units_allowed,-1,-1;category "category" true true false 512 Text 0 0,First,#,residential_parcels,category,0,512', 
    ''
)

<Result 'shp\\Missoula.gdb\\missoula_residential_parcels'>